In [1]:
import pickle
#from loadData import dataLoader
import tensorflow as tf
#from CalculateEER import SKGetEER as GetEER
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_curve, auc
from scipy import interp


In [54]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t

In [168]:
def GetEER(y_score, y_test):
    #y_score = y_score.get_label()
    y_test = np.array(y_test.get_label()).reshape(len(y_score), 1)
    n_classes = y_score.shape[1]
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    missRate = dict()
    #print(y_test)
    #print([np.argmax(x) for x in y for y in y_score])
    #y_test_ = [onehot(int(x[0]), 100) for x in y_test]
    y_score_ = [onehot(x,100) for x in y_score]
    #print(np.shape(y_score), np.shape(y_test))
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_,y_score)
        roc_auc[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    missRate["micro"] = 1 - tpr["micro"]
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= n_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    missRate["macro"] = 1 - tpr["macro"]
    #print(np.dtype(min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])))
    return 'EER', min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])


In [182]:
def GetEER_(y_score, y_test):
	n_classes = y_score.shape[1]
	fpr = dict()
	tpr = dict()
	roc_auc = dict()
	missRate = dict()
	for i in range(n_classes):
		fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
		roc_auc[i] = auc(fpr[i], tpr[i])
	fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
	missRate["micro"] = 1 - tpr["micro"]
	all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
	# Then interpolate all ROC curves at this points
	mean_tpr = np.zeros_like(all_fpr)
	for i in range(n_classes):
		mean_tpr += interp(all_fpr, fpr[i], tpr[i])
	mean_tpr /= n_classes
	fpr["macro"] = all_fpr
	tpr["macro"] = mean_tpr
	missRate["macro"] = 1 - tpr["macro"]
	return  100*min(fpr["micro"][np.argmin(abs(fpr["micro"]-missRate["micro"]))], fpr["macro"][np.argmin(abs(fpr["macro"]-missRate["macro"]))])


In [178]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
data= pickle.load(open('/Users/chaitanya/Documents/python/keystrokes/data_augmentation/greyc1/PCA_GREYC_2X.pickle','rb'))


In [203]:
X = data['data']
X = np.array(X).reshape(len(X), 28)
y = np.array(data['labels']).reshape(len(X), 100)
#y_train = [onehot(x,100) for x in train_data['labels']]
#X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
#y_test = [onehot(x,100) for x in test_data['labels']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=12)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=122)
print([len(x) for x in [X_test, y_test, X_train, y_train]])

#y_train=np.reshape(y_train,(len(y_train)*100))
#y_test = np.reshape(y_test, (len(y_test)*100))

[2223, 2223, 12593, 12593]


In [209]:
y_train_ = [np.argmax(x) for x in y_train]
y_test_ = [np.argmax(x) for x in y_test]
xg_train = xgb.DMatrix(X_train, label = y_train_)
xg_test = xgb.DMatrix(X_test, label =  y_test_)
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.1
param['max_depth'] = 4
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 100
param['n_estimators'] = 1000
#param['nrounds'] = 
param['tree_method']= 'gpu_hist'

n_classes = 100
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 100
bst = xgb.train(param, xg_train, num_round,  watchlist)#, feval = GetEER, maximize = False)



XGBoostError: b'[00:40:38] src/learner.cc:153: XGBoost version not compiled with GPU support.\n\nStack trace returned 2 entries:\n[bt] (0) 0   libxgboost.dylib                    0x000000011dc63683 dmlc::StackTrace[abi:cxx11]() + 67\n[bt] (1) 1   libstdc++.6.dylib                   0x000000011e0e2ce0 vtable for std::__cxx11::basic_stringbuf<char, std::char_traits<char>, std::allocator<char> > + 16\n\n'

In [205]:
#ytest = np.reshape(y_test,(len(X_test), 100))
pred = bst.predict(xg_test)
error_rate = np.sum(np.argmax(pred) != np.argmax(y_test))/np.shape(y_test)[0]

In [206]:
1-error_rate

0.9995501574448943

In [207]:
GetEER_(pred,y_test)


2.235581182949604

In [163]:
np.shape(pred), np.shape(y_test)

((4445, 100), (4445, 100))

In [146]:
np.argmax(pred[3])

73

In [202]:
xg_train.get_label()

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [201]:
len(pred.ravel())

444500

In [ ]:
xg_train